##Pre

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#nos ayuda para importar pdfs
!pip install textract


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 83.9 MB/s 
     |████████████████████████████████| 69 kB 8.8 MB/s 
     |████████████████████████████████| 106 kB 70.7 MB/s 
     |████████████████████████████████| 10.1 MB 42.7 MB/s 
     |████████████████████████████████| 5.6 MB 26.6 MB/s 
     |████████████████████████████████| 2.3 MB 63.8 MB/s 
     |████████████████████████████████| 128 kB 80.9 MB/s 
     |████████████████████████████████| 112 kB 57.1 MB/s 
     |████████████████████████████████| 73 kB 3.5 MB/s 
     |████████████████████████████████| 149 kB 80.0 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
     |████████████████████████████████| 340 kB 79.2 MB/s 
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3980 sha256=96329c5a9be58ef49a2057f3198c03c2100b2ef500faf90da0223f1c1e4fcb46
  Stored in directory: /root/.cache/pip/wheels/55/f0/2c/81

In [ ]:
import textract
import pandas as pd

In [ ]:
#importando el pdf en txt
text = textract.process('/content/drive/MyDrive/data_set_entrenamiento/NLP/4.pdf', method='pdfminer', encoding='ascii')

In [ ]:
#separar por paginas '\x0c'
list_text_inicial = text.decode().split('\x0c')
list_text_inicial


[' \n\n \n\nForward the Foundation\n\nBook 2 in the Foundation Series\n\nIsaac Asimov\n\n',
 ' For nearly fifty years, Isaac Asimov thrilled millions of readers with his internationally \nbestselling Foundation Series, a spell-binding tale of the future that spans hundreds of \nyears and dozens of worlds. Here, now, is Forward the Foundation, the seventh and final \nvolume in the series. Completed just before his death, it is the Grand Master’s last gift to \nhis legions of admirers.\n\nHere, at last, is the story Asimov fans have been waiting for, an exciting tale of danger, \nintrigue, and suspense that chronicles the second half of hero Hari Seldon’s life as he \nstruggles to perfect his revolutionary Theory of Psychohistory and establish the means \nby which the survival of humanity will be ensured: Foundation. For, as Seldon and his \nloyal band of followers know, the mighty Galactic Empire is crumbling, and its \ninevitable destruction will wreak havoc Galaxy-wide ...\n\nA resoun

In [ ]:
#separar los textos por parrafos \n\n con un for
text_total = []
for page in list_text_inicial[4:]:
    text_total.append(page.split('\n\n'))


In [ ]:
#ocupar un replace para quitar los saltos de linea
text_total_new = []
for pagina in text_total:
    page = []
    for parrafo in pagina:
        if len(parrafo)>0:
            page.append(parrafo.replace('\n',' '))
    text_total_new.append(page)


In [ ]:
!pip install contractions
import contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 43.2 MB/s 
     |████████████████████████████████| 110 kB 76.6 MB/s 


In [ ]:
import re

In [ ]:
#Limpieza de todos los simbolos que no son alfa-numericos con re.sub
#usar contractions.fix para quitar contracciones
#y .lower para pasar todo a minusculas
new_list_clean = []
for page in text_total_new:
    page_new = []
    for paragraph in page:
        if len(paragraph.split())>0:
            page_new.append(re.sub(r'[\W_]+',' ',contractions.fix(paragraph)).replace(' s ',' ').lower())
    new_list_clean.append(page_new)
 

In [ ]:
#Generando los data frames:
#
#Corpus

#Stemming y lematization: llevar las palabras a su 'raiz'
#que de la mayor informacion 


df_tex_uno = pd.DataFrame(new_list_clean)



In [ ]:
#quitar stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Quitando stopwords con un for 
#

final_text_uno = []
for page in new_list_clean:
    page_aux = []
    for paragraph in page:
        text=[]
        for word in word_tokenize(paragraph):
            if word not in stopwords:
                text.append(word)
        page_aux.append(' '.join(text))
    final_text_uno.append(page_aux)


In [ ]:
#generar los dataframes sin stopwords 

#df_sinword_uno
total_text_uno = []
for num,page in enumerate(final_text_uno):
    for paragraph in page:
        total_text_uno.append((paragraph,num+1))


df_sinword_uno = pd.DataFrame(total_text_uno).rename(columns={0:'parrafos',1:'pagina'})       

##Ejercicio 1.
Utilice el los corpus del ejercicio 4 de la tarea de preprocesamiento para obtener
un CBOW embeding. Guarde los pesos en un txt.

In [ ]:
import tensorflow as tf
from keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True 
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [ ]:
from keras.layers import Embedding, Dense, Lambda 
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
import nltk

In [ ]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
remove_terms = punctuation + '0123456789'

In [ ]:
def preprocessing(string):
    words = word_tokenize(string)
    tokens = [w for w in words if w.lower() not in remove_terms]
    stopw = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopw ]
    tokens = [word for word in  tokens if len(word)>=3]
    tokens = [word for word in tokens if word.isalpha()]
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens]
    preprocess_text = ' '.join(tokens)
    return preprocess_text

In [ ]:
#corpus = open(total_text_uno,encoding='utf8' ).readlines()

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
corpus_uno = [preprocessing(sentence) for sentence in df_sinword_uno['parrafos'] if sentence.strip()!='']
corpus_uno

['',
 'tell hari said yugo amaryl friend demerzel deep trouble emphasized word friend lightly unmistakable air distaste',
 'hari seldon detected sour note ignored looked tricomputer said tell yugo nonsense trace annoyance trace added taking time insisting',
 'think important amaryl sat defiantly gesture indicated going moved easily would stay',
 'eight year heatsinker dahl sector low social scale possible lifted position seldon made mathematician intellectual psychohistorian',
 'never one minute forget owed change meant speak harshly hari seldon seldon good consideration respect love older man regard career would stop owed harshness much seldon',
 'look hari said chopping air left hand reason beyond understanding think highly demerzel one whose opinion respect except think well care happens personally hari long thinkyou choice bring attention',
 'seldon smiled much earnestness considered uselessness concern fond yugo amaryl fond yugo one four people encountered short period life flight

In [ ]:
tokeneizer = Tokenizer()
tokeneizer.fit_on_texts(corpus_uno)

In [ ]:
X_train_tokens = tokeneizer.texts_to_sequences(corpus_uno)

In [ ]:
X_train_tokens

[[],
 [33, 8, 2, 63, 51, 169, 38, 548, 176, 2860, 97, 169, 2264, 2265, 362, 2861],
 [8,
  1,
  2862,
  2863,
  1038,
  1408,
  55,
  2864,
  2,
  33,
  63,
  890,
  1039,
  1910,
  1039,
  824,
  344,
  10,
  3974],
 [11, 221, 51, 200, 3975, 1040, 2266, 44, 406, 489, 3, 453],
 [196,
  15,
  1911,
  212,
  143,
  691,
  891,
  1912,
  165,
  825,
  387,
  1,
  62,
  285,
  1261,
  1150],
 [52,
  4,
  407,
  577,
  2865,
  201,
  609,
  230,
  1913,
  8,
  1,
  1,
  69,
  1914,
  892,
  408,
  374,
  22,
  2267,
  1409,
  3,
  345,
  2865,
  3976,
  45,
  1],
 [75,
  8,
  2,
  3977,
  362,
  100,
  57,
  170,
  610,
  962,
  11,
  1262,
  38,
  4,
  777,
  655,
  892,
  293,
  11,
  13,
  222,
  518,
  1410,
  8,
  89,
  3978,
  826,
  325,
  827],
 [1,
  180,
  45,
  3979,
  893,
  2268,
  490,
  1411,
  63,
  51,
  1411,
  63,
  4,
  429,
  26,
  1915,
  409,
  828,
  80,
  2866,
  549,
  84,
  388,
  32,
  550,
  38,
  9,
  181,
  63,
  51,
  6,
  429,
  20,
  148,
  102],
 [551,
  11

In [ ]:
word2id = tokeneizer.word_index
id2word = dict([(value,key) for (key,value) in word2id.items() ])

In [ ]:
vocab_size = len(word2id)+1
vocab_size
embed_size = 300
window_size = 2

In [ ]:
def generate_context_word_pairs(corpus, window_size,vocab_size):
    context_lenght = window_size*2
    for words in corpus:
        sentence_lenght = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word = []
            start = index-window_size
            end = index + window_size + 1
            context_words.append([words[i]
                                 for i in range(start,end)
                                 if 0<=i<sentence_lenght and i!=index])
            label_word.append(word)
            
            x = pad_sequences(context_words, maxlen= context_lenght)
            y = to_categorical(label_word,vocab_size)
            yield (x,y)

In [ ]:
i=0
for x,y in generate_context_word_pairs(corpus=X_train_tokens, window_size=window_size,vocab_size=vocab_size):
    if 0 not in x[0]:
        print('Context (X):',[id2word[w] for w in x[0]],'-> Target (Y):', id2word[np.argwhere(y[0])[0][0]] )
    if i == 1000:
        break
    i+=1 

Context (X): ['tell', 'hari', 'yugo', 'amaryl'] -> Target (Y): said
Context (X): ['hari', 'said', 'amaryl', 'friend'] -> Target (Y): yugo
Context (X): ['said', 'yugo', 'friend', 'demerzel'] -> Target (Y): amaryl
Context (X): ['yugo', 'amaryl', 'demerzel', 'deep'] -> Target (Y): friend
Context (X): ['amaryl', 'friend', 'deep', 'trouble'] -> Target (Y): demerzel
Context (X): ['friend', 'demerzel', 'trouble', 'emphasized'] -> Target (Y): deep
Context (X): ['demerzel', 'deep', 'emphasized', 'word'] -> Target (Y): trouble
Context (X): ['deep', 'trouble', 'word', 'friend'] -> Target (Y): emphasized
Context (X): ['trouble', 'emphasized', 'friend', 'lightly'] -> Target (Y): word
Context (X): ['emphasized', 'word', 'lightly', 'unmistakable'] -> Target (Y): friend
Context (X): ['word', 'friend', 'unmistakable', 'air'] -> Target (Y): lightly
Context (X): ['friend', 'lightly', 'air', 'distaste'] -> Target (Y): unmistakable
Context (X): ['hari', 'seldon', 'sour', 'note'] -> Target (Y): detected
Con

In [ ]:
feactures = generate_context_word_pairs(corpus=X_train_tokens[:20], window_size=window_size,vocab_size=vocab_size)

In [ ]:
for x,y in feactures:
    print(x[0])
    print('-----')

[0 0 8 2]
-----
[ 0 33  2 63]
-----
[33  8 63 51]
-----
[  8   2  51 169]
-----
[  2  63 169  38]
-----
[ 63  51  38 548]
-----
[ 51 169 548 176]
-----
[ 169   38  176 2860]
-----
[  38  548 2860   97]
-----
[548 176  97 169]
-----
[ 176 2860  169 2264]
-----
[2860   97 2264 2265]
-----
[  97  169 2265  362]
-----
[ 169 2264  362 2861]
-----
[   0 2264 2265 2861]
-----
[   0    0 2265  362]
-----
[   0    0    1 2862]
-----
[   0    8 2862 2863]
-----
[   8    1 2863 1038]
-----
[   1 2862 1038 1408]
-----
[2862 2863 1408   55]
-----
[2863 1038   55 2864]
-----
[1038 1408 2864    2]
-----
[1408   55    2   33]
-----
[  55 2864   33   63]
-----
[2864    2   63  890]
-----
[   2   33  890 1039]
-----
[  33   63 1039 1910]
-----
[  63  890 1910 1039]
-----
[ 890 1039 1039  824]
-----
[1039 1910  824  344]
-----
[1910 1039  344   10]
-----
[1039  824   10 3974]
-----
[   0  824  344 3974]
-----
[  0   0 344  10]
-----
[  0   0 221  51]
-----
[  0  11  51 200]
-----
[  11  221  200 3975]
--

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                  output_dim=embed_size,
                  embeddings_initializer='glorot_uniform',
                  input_length = window_size*2))
model.add(Lambda(lambda x: K.mean(x,axis=1), output_shape = (embed_size,)))
model.add(Dense(vocab_size,kernel_initializer='glorot_uniform', activation= 'softmax'))
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 300)            2244300   
                                                                 
 lambda (Lambda)             (None, 300)               0         
                                                                 
 dense (Dense)               (None, 7481)              2251781   
                                                                 
Total params: 4,496,081
Trainable params: 4,496,081
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
%%time
n_epochs = 10
for epoch in  range(n_epochs):
    loss = 0
    for x,y in generate_context_word_pairs(corpus=X_train_tokens, window_size=window_size,vocab_size=vocab_size):
        loss += model.train_on_batch(x,y)
        
    print('Epoch:',epoch, '\tLoss:', loss)

Epoch: 0 	Loss: 381.0758342923946
Epoch: 1 	Loss: 81.38763703048608
Epoch: 2 	Loss: 67.95753155355123
Epoch: 3 	Loss: 62.649134166988006
Epoch: 4 	Loss: 57.41010472369089
Epoch: 5 	Loss: 52.840915317312465
Epoch: 6 	Loss: 48.74034902984931
Epoch: 7 	Loss: 44.94457450056507
Epoch: 8 	Loss: 40.745050943293336
Epoch: 9 	Loss: 36.543670064606886
CPU times: user 59min 15s, sys: 5min 33s, total: 1h 4min 48s
Wall time: 1h 5s


In [ ]:
weights = model.get_weights()[0]
items = tokeneizer.word_index.items()

f = open('/content/drive/MyDrive/data_set_entrenamiento/NLP/cbow_forward.txt','w',encoding='utf8')
f.write('{} {}\n'.format(vocab_size-1,300))

weights = model.get_weights()[0]
for word, i in items:
    f.write('{} {}\n'.format(word, ' '.join(map(str,list(weights[i,:])))))
f.close()

In [ ]:
df = pd.DataFrame(weights[1:],index=list(id2word.values()))

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
seldon,-1.510377,-2.211530,-0.455582,-1.106469,0.506097,-1.474920,-0.699581,-1.332498,0.857182,0.834852,...,0.716413,-0.795343,-2.598911,0.630420,1.477722,0.552468,-0.593329,0.684731,0.819297,0.625585
said,2.554572,1.834167,1.061377,-0.004430,0.340326,-1.594573,-0.593626,-1.355778,-0.997595,-0.209862,...,1.510902,-0.966457,-2.383356,-1.160290,-1.958798,-0.362746,-0.017070,1.398151,-1.399987,-1.438785
would,-0.003946,-0.672490,0.179971,0.739482,-1.506995,-2.058109,1.796500,-1.151899,-0.615467,1.736063,...,-0.502104,0.600406,0.158926,-0.745432,-0.170949,-0.689842,-0.952176,-0.415430,-0.219410,-1.727659
one,2.030361,2.231172,0.539184,-0.166121,0.885400,1.103419,-1.828709,-1.527326,-0.403985,0.595627,...,-1.217918,-0.755913,-1.065693,-1.416099,1.550595,-0.120409,-1.409878,-0.460838,-0.181083,0.116753
know,0.699646,-1.122353,-1.593386,2.225462,-0.751079,-0.408778,0.874755,1.409812,-0.757816,-0.900460,...,-0.495155,0.874840,-0.473593,-0.583477,-1.161873,-0.743471,0.568008,0.736413,0.684126,0.277624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
saga,0.157807,-0.015674,-0.277489,0.064080,-0.338243,0.406026,-0.293226,0.181100,-0.019570,0.044104,...,0.100338,-0.203860,-0.152553,0.156263,0.141631,0.255468,0.174491,-0.114099,-0.084364,-0.321512
writer,0.226584,0.161120,-0.205775,0.068272,-0.289934,0.131726,-0.180200,0.163591,-0.144847,0.085146,...,0.000723,-0.154131,-0.236214,0.067889,0.252600,0.287082,0.290535,-0.059999,-0.036107,-0.233287
america,0.325272,0.153950,-0.302856,-0.033951,-0.302416,0.221611,-0.264160,0.078009,-0.137437,-0.061504,...,-0.037806,-0.337017,-0.315873,-0.021726,0.273936,0.285547,0.248862,0.039003,-0.282129,-0.323527
entertained,0.276291,0.289215,-0.331294,-0.123010,-0.334150,0.175832,-0.341424,0.206543,-0.011782,-0.323697,...,-0.315497,-0.392264,-0.405177,-0.314060,0.392785,0.377428,0.363236,-0.182506,-0.328638,-0.317244


##Ejercicio dos

In [ ]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input
from keras.layers import Dot
#from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
import gensim 

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus_uno)

vocab_size = len(tokenizer.word_index)+1
vocab_size
items = tokenizer.word_index.items()

In [ ]:
dim_embeddings = 300

inputs = Input(shape=(1,),dtype='int32')
w = Embedding(vocab_size, dim_embeddings)(inputs)

c_inputs = Input(shape=(1,),dtype='int32')
c = Embedding(vocab_size, dim_embeddings)(c_inputs)

d = Dot(axes=2)([w,c])

d = Reshape((1,),input_shape=(1,1))(d)
d = Activation('sigmoid')(d)

model = Model(inputs=[inputs,c_inputs],outputs=d)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 300)       2244300     ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, 1, 300)       2244300     ['input_2[0][0]']                
                                                                                              

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam')

In [ ]:
enumerate(X_train_tokens)

In [ ]:
for _, doc in enumerate(X_train_tokens):
    print(doc)

[]
[33, 8, 2, 63, 51, 169, 38, 548, 176, 2860, 97, 169, 2264, 2265, 362, 2861]
[8, 1, 2862, 2863, 1038, 1408, 55, 2864, 2, 33, 63, 890, 1039, 1910, 1039, 824, 344, 10, 3974]
[11, 221, 51, 200, 3975, 1040, 2266, 44, 406, 489, 3, 453]
[196, 15, 1911, 212, 143, 691, 891, 1912, 165, 825, 387, 1, 62, 285, 1261, 1150]
[52, 4, 407, 577, 2865, 201, 609, 230, 1913, 8, 1, 1, 69, 1914, 892, 408, 374, 22, 2267, 1409, 3, 345, 2865, 3976, 45, 1]
[75, 8, 2, 3977, 362, 100, 57, 170, 610, 962, 11, 1262, 38, 4, 777, 655, 892, 293, 11, 13, 222, 518, 1410, 8, 89, 3978, 826, 325, 827]
[1, 180, 45, 3979, 893, 2268, 490, 1411, 63, 51, 1411, 63, 4, 429, 26, 1915, 409, 828, 80, 2866, 549, 84, 388, 32, 550, 38, 9, 181, 63, 51, 6, 429, 20, 148, 102]
[551, 111, 242, 23, 429, 2867, 63, 51, 963, 962, 1151, 14, 3980, 3981, 82, 454, 3982, 5, 73, 184, 1, 468, 552, 7, 611, 256, 346, 2868, 3983, 3984]
[3, 153, 469, 4, 69, 70, 3, 363, 519]
[2, 326, 63, 76, 1916, 3985, 57, 305, 2869, 31, 156, 202, 894, 491, 964, 87]
[51, 

In [ ]:
skipgrams(sequence=doc, vocabulary_size = vocab_size, window_size=4)

([[3545, 5224],
  [751, 50],
  [2812, 271],
  [369, 3972],
  [2812, 6768],
  [3, 2593],
  [435, 3583],
  [522, 1350],
  [1893, 331],
  [2812, 4],
  [1803, 3250],
  [3250, 5585],
  [522, 2812],
  [4, 3809],
  [7474, 5864],
  [1803, 7476],
  [2812, 331],
  [289, 6445],
  [4, 6409],
  [3545, 1803],
  [1409, 2332],
  [7474, 7473],
  [7476, 147],
  [24, 7476],
  [1409, 5298],
  [1803, 7095],
  [7473, 4916],
  [331, 2303],
  [522, 5228],
  [626, 522],
  [522, 3612],
  [286, 5997],
  [4, 7394],
  [7477, 2263],
  [1803, 7204],
  [1126, 409],
  [893, 3],
  [1803, 1409],
  [522, 2263],
  [24, 1803],
  [331, 751],
  [1803, 522],
  [3973, 317],
  [271, 2362],
  [1803, 5642],
  [147, 4182],
  [2263, 7473],
  [3972, 3083],
  [317, 2812],
  [2400, 522],
  [522, 6380],
  [7472, 1409],
  [24, 956],
  [2400, 7439],
  [369, 5398],
  [1803, 522],
  [59, 1803],
  [4, 289],
  [1803, 510],
  [1803, 7479],
  [331, 1893],
  [2263, 7477],
  [50, 717],
  [2812, 3545],
  [626, 2263],
  [522, 2368],
  [1350, 4863]

In [ ]:
%%time
n_epochs = 20
for epoch in range(n_epochs):
    loss = 0 
    for _, doc in enumerate(X_train_tokens):
        data, labels = skipgrams(sequence=doc, vocabulary_size = vocab_size, window_size=4)
        x = [np.array(x) for x in zip(*data)]
        y = np.array(labels,dtype=np.int32)
        if x:
            loss += model.train_on_batch(x,y)
            
    print('Epoch: ', epoch, '\tloss:',loss)

Epoch:  0 	loss: 2274.7147668525577
Epoch:  1 	loss: 1775.023648250848
Epoch:  2 	loss: 1601.455164488405
Epoch:  3 	loss: 1422.8093133801594
Epoch:  4 	loss: 1253.9840636891313
Epoch:  5 	loss: 1114.560769641772
Epoch:  6 	loss: 982.5078625429887
Epoch:  7 	loss: 897.4467993719736
Epoch:  8 	loss: 728.1940710972995
Epoch:  9 	loss: 652.779714690223
Epoch:  10 	loss: 598.4441158218979
Epoch:  11 	loss: 553.7711214442897
Epoch:  12 	loss: 523.1464925769269
Epoch:  13 	loss: 496.089377126882
Epoch:  14 	loss: 479.6767867581184
Epoch:  15 	loss: 442.42285709672706
Epoch:  16 	loss: 435.0393488750092
Epoch:  17 	loss: 421.2727744824301
Epoch:  18 	loss: 405.55023359621373
Epoch:  19 	loss: 400.1134299725185
CPU times: user 12min 39s, sys: 56.4 s, total: 13min 35s
Wall time: 11min 53s


In [ ]:
weights = model.get_weights()[0]
items = tokenizer.word_index.items()

f = open('/content/drive/MyDrive/data_set_entrenamiento/NLP/skip_forward.txt','w',encoding='utf8')
f.write('{} {}\n'.format(vocab_size-1,300))

weights = model.get_weights()[0]
for word, i in items:
    f.write('{} {}\n'.format(word, ' '.join(map(str,list(weights[i,:])))))
f.close()

In [ ]:
df_skip = pd.DataFrame(weights[1:],index=list(id2word.values()))

In [ ]:
df_skip


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
seldon,0.007226,-0.111685,-0.308369,-0.031631,-0.317851,-0.042278,0.250395,-0.017808,0.117106,0.289152,...,0.073192,0.053944,0.034914,0.101762,-0.080692,0.061780,0.045041,0.398819,0.160707,-0.104580
said,-0.096569,-0.241980,-0.035588,0.002086,0.121016,0.020931,0.015131,-0.028094,0.185375,-0.005358,...,0.045841,-0.026400,0.011429,0.111553,-0.119750,0.507413,0.270912,0.043111,-0.022769,0.108721
would,-0.265114,-0.090757,0.039142,-0.095847,-0.468200,-0.024579,-0.145122,0.011788,-0.074260,0.007267,...,0.175623,0.317392,-0.008971,-0.065455,0.002117,-0.134044,0.032848,0.153830,-0.372617,0.050044
one,-0.019693,-0.006234,0.129650,0.014374,-0.204127,-0.021905,0.124372,-0.043674,0.037554,0.092641,...,0.229378,-0.091773,0.126605,0.000282,-0.102223,0.155743,0.481115,-0.127262,-0.074646,-0.115258
know,-0.298476,0.009425,0.098728,0.016940,-0.275460,-0.046573,-0.094654,0.101946,-0.261315,0.271063,...,0.042739,-0.009139,0.174852,0.181841,-0.080577,-0.131348,0.197439,0.088139,0.385947,-0.362730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
saga,0.242268,0.097869,-0.031078,-0.106423,0.040026,0.045246,0.045210,0.125719,-0.024259,-0.103501,...,-0.071040,-0.485333,-0.013807,0.242250,-0.125083,0.020674,0.107292,0.009104,-0.155830,0.079758
writer,0.117332,0.230369,0.086693,0.071964,0.018933,0.071089,0.154498,0.056194,-0.093364,0.034952,...,-0.001066,-0.343981,0.048096,-0.045607,0.000704,-0.032389,-0.042339,-0.083042,0.067133,-0.083752
america,-0.003119,-0.105320,0.000314,-0.026235,-0.078930,0.070843,0.136322,0.073636,-0.106653,0.055928,...,-0.045240,-0.176465,0.070261,0.080991,0.061182,0.020479,-0.039971,-0.085409,0.264510,0.012275
entertained,0.069967,-0.142798,-0.024210,0.042395,-0.024737,0.015996,0.026917,-0.032566,-0.005383,0.019161,...,-0.051848,-0.246984,0.086839,-0.054231,0.036660,-0.007066,-0.043561,0.004672,-0.011268,-0.006504
